### Ingest lap_times folder

##### Step 1 - Read the CSV file using the spark dataframe reader API

In [7]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [8]:
lap_times_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), True),
                                      StructField("lap", IntegerType(), True),
                                      StructField("position", IntegerType(), True),
                                      StructField("time", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True)
                                     ])

In [9]:
lap_times_df = spark.read \
.schema(lap_times_schema) \
.csv("/mnt/gualterformula1dl/raw/lap_times")

##### Step 2 - Rename columns and add new columns
1. Rename driverId and raceId
1. Add ingestion_date with current timestamp

In [10]:
from pyspark.sql.functions import current_timestamp

In [11]:
final_df = lap_times_df.withColumnRenamed("driverId", "driver_id") \
.withColumnRenamed("raceId", "race_id") \
.withColumn("ingestion_date", current_timestamp())

##### Step 3 - Write to output to processed container in parquet format

In [12]:
final_df.write.mode("overwrite").parquet("/mnt/gualterformula1dl/processed/lap_times")